# qmrpy チュートリアル（機能網羅）

このノートブックは **qmrpy の主要機能を一通り触るための最小例** です。
実運用では `configs/` や `scripts/` を使った再現可能な run を推奨します。


## 0. セットアップ

`uv` を使う場合：

```bash
uv add qmrpy
```

可視化を行う場合は `plotnine` を使います（`qmrpy[viz]` を追加）。


In [ ]:
import numpy as np

# 乱数を固定
rng = np.random.default_rng(0)


## 1. T1: VFA（Variable Flip Angle）


In [ ]:
from qmrpy.models.t1.vfa_t1 import VfaT1

model_vfa = VfaT1(flip_angle_deg=[2, 5, 10, 15], tr_s=0.015)
signal_vfa = model_vfa.forward(m0=1.0, t1_s=1.2)
fit_vfa = model_vfa.fit_linear(signal_vfa)

signal_vfa, fit_vfa

# 可視化


## 2. T1: Inversion Recovery


In [ ]:
from qmrpy.models.t1.inversion_recovery import InversionRecovery

ti_ms = np.array([50, 100, 200, 400, 800, 1200])
model_ir = InversionRecovery(ti_ms=ti_ms)
signal_ir = model_ir.forward(t1_ms=900.0, ra=500.0, rb=-1000.0, magnitude=True)
fit_ir = model_ir.fit(signal_ir, method="magnitude")

signal_ir, fit_ir



## 3. T2: Mono-exponential


In [ ]:
from qmrpy.models.t2.mono_t2 import MonoT2

te_ms = np.arange(10, 110, 10)
model_t2 = MonoT2(te=te_ms)
signal_t2 = model_t2.forward(m0=1.0, t2=60.0)
fit_t2 = model_t2.fit(signal_t2, fit_type="exponential")

signal_t2, fit_t2



## 4. T2: Multi-component（MWF）


In [ ]:
from qmrpy.models.t2.mwf import MultiComponentT2

te_ms = np.arange(10, 330, 10)
model_mwf = MultiComponentT2(te_ms=te_ms)

# 2成分の簡単な合成（basisに合わせた重み）
basis = model_mwf.t2_basis_ms
weights = np.zeros_like(basis)
weights[np.argmin(np.abs(basis - 20.0))] = 0.15
weights[np.argmin(np.abs(basis - 80.0))] = 0.85

signal_mwf = model_mwf.forward(weights=weights)
fit_mwf = model_mwf.fit(signal_mwf)

fit_mwf



## 5. T2: DECAES T2 Map（EPG + NNLS）

小さなETLで合成信号を作り、`DecaesT2Map.fit` を試します。


In [ ]:
from qmrpy.models.t2.decaes_t2 import DecaesT2Map, epg_decay_curve

n_te = 8
te_s = 0.01
model_decaes = DecaesT2Map(
    n_te=n_te,
    te_s=te_s,
    n_t2=20,
    t2_range_s=(0.01, 1.0),
    t1_s=1.0,
    set_flip_angle_deg=180.0,
    reg="none",
)

# 2成分の減衰を重み付き合成
decay_fast = epg_decay_curve(etl=n_te, alpha_deg=180.0, te_s=te_s, t2_s=0.03, t1_s=1.0, beta_deg=180.0)
decay_slow = epg_decay_curve(etl=n_te, alpha_deg=180.0, te_s=te_s, t2_s=0.12, t1_s=1.0, beta_deg=180.0)
signal_decaes = 0.2 * decay_fast + 0.8 * decay_slow

fit_decaes = model_decaes.fit(signal_decaes)
fit_decaes.keys()



## 6. T2: DECAES T2 Part（分布→分画）


In [ ]:
from qmrpy.models.t2.decaes_t2part import DecaesT2Part

model_t2part = DecaesT2Part(
    n_t2=20,
    t2_range_s=(0.01, 1.0),
    spwin_s=(0.01, 0.05),
    mpwin_s=(0.05, 0.2),
)

# fit_decaes["distribution"] が distribution です（DecaesT2Map.fit の出力）
dist = fit_decaes["distribution"]
model_t2part.fit(dist)

t2_s = model_t2part.t2_times_s()


## 7. B1: DAM（Double-Angle Method）


In [ ]:
from qmrpy.models.b1.dam import B1Dam

model_b1 = B1Dam(alpha_deg=60.0)
signal_b1 = model_b1.forward(m0=1.0, b1=1.1)
fit_b1 = model_b1.fit_raw(signal_b1)

signal_b1, fit_b1



## 8. ノイズモデル（Gaussian / Rician）


In [ ]:
from qmrpy.sim.noise import add_gaussian_noise, add_rician_noise

signal = np.linspace(0.2, 1.0, 6)
g = add_gaussian_noise(signal, sigma=0.05, rng=rng)
r = add_rician_noise(signal, sigma=0.05, rng=rng)
signal, g, r



## 9. ノイズ低減: MPPCA


In [ ]:
from qmrpy.models.noise.denoising_mppca import MPPCA

# 4D (x, y, z, t) の小さなデータ
data = rng.normal(0.0, 1.0, size=(6, 6, 1, 8))
mppca = MPPCA(kernel=(3, 3, 1))
denoise = mppca.fit(data)
denoise.keys()

# 1ボクセルの時系列を例示
vox = data[0, 0, 0, :]
vox_d = denoise["denoised"][0, 0, 0, :]


## 10. シミュレーション: SingleVoxel / SimVary / SimRnd


In [ ]:
from qmrpy.sim.simulation import simulate_single_voxel, sensitivity_analysis, simulate_parameter_distribution

# SingleVoxel
single = simulate_single_voxel(
    model_vfa,
    params={"m0": 1.0, "t1_s": 1.2},
    noise_model="gaussian",
    noise_sigma=0.01,
    rng=rng,
    fit=True,
)
single.keys()



In [ ]:
# SimVary
vary = sensitivity_analysis(
    model_vfa,
    nominal_params={"m0": 1.0, "t1_s": 1.2},
    vary_param="t1_s",
    lb=0.5,
    ub=2.0,
    n_steps=6,
    n_runs=5,
    noise_model="gaussian",
    noise_sigma=0.01,
    rng=rng,
)
vary["mean"].keys()



In [ ]:
# SimRnd (parameter distribution)
t1_samples = rng.uniform(0.6, 1.6, size=20)
simrnd = simulate_parameter_distribution(
    model_vfa,
    true_params={"m0": 1.0, "t1_s": t1_samples},
    noise_model="gaussian",
    noise_sigma=0.01,
    rng=rng,
)
simrnd["metrics"]



## 11. Fisher / CRLB / Protocol最適化


In [ ]:
from qmrpy.sim.simulation import fisher_information_gaussian, crlb_from_fisher, optimize_protocol_grid

# Fisher & CRLB (VFAの例)
def model_factory(protocol):
    return VfaT1(flip_angle_deg=protocol, tr_s=0.015)

model = VfaT1(flip_angle_deg=[2, 5, 10, 15], tr_s=0.015)
fisher = fisher_information_gaussian(model, params={"m0": 1.0, "t1_s": 1.2}, sigma=0.01)
crlb = crlb_from_fisher(fisher)
crlb

# crlb は ndarray の場合があるので分岐
if hasattr(crlb, "keys"):
else:


In [ ]:
# Protocol grid search (例: flip angle 候補から最小CRLBを探す)
protocol_candidates = [
    [2, 5, 10, 15],
    [3, 8, 13, 18],
    [5, 10, 20, 30],
]

opt = optimize_protocol_grid(
    model_factory=model_factory,
    protocol_candidates=protocol_candidates,
    params={"m0": 1.0, "t1_s": 1.2},
    sigma=0.01,
)
opt



## 12. 4D Phantom 生成（簡易）


In [ ]:
from qmrpy.sim.phantoms import generate_4d_phantom

phantom, truth, sigma = generate_4d_phantom(sx=8, sy=8, sz=2, n_vol=6, snr=20.0, seed=0)
phantom.shape, truth.shape, sigma

# 1ボクセルの時系列を可視化
vox = phantom[0, 0, 0, :]
